In [1]:
import numpy as np

from kaggle_movie_genres.config import load_config
from kaggle_movie_genres.labelhandler import LabelHandler
from kaggle_movie_genres.featurizer import create_tokenizer_and_embedder
from kaggle_movie_genres.dataloader import create_dataloader
from kaggle_movie_genres.cls_classifier import CLS_Classifier
from kaggle_movie_genres.submission import format_predictions
from kaggle_movie_genres.trainpredict import TrainPredict
import logging
logging.basicConfig(level=logging.INFO,    
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S'  # Date + short timestamp
)
logger = logging.getLogger(__name__)
import torch.nn as nn
import torch
import tqdm


### Helper stuff    

In [2]:
# config contains all constants pathes and settings
config = load_config()

# label_handler helps to convert labels between different formats
label_handler = LabelHandler(config)



### Tokenizer, embedder and the model

In [3]:
tokenizer, embedder = create_tokenizer_and_embedder(config)
model = CLS_Classifier(embedder, num_labels=label_handler.get_multi_hot_length(), config=config)

### Create train / validation sets

In [4]:
train_set, validation_set = create_dataloader('data/train.csv', tokenizer, label_handler, config, validation_split=True)
test_set,_ = create_dataloader('data/test.csv', tokenizer, label_handler, config, validation_split=False)

2025-11-15 23:51:58 - INFO - Loaded 8000 records from data/train.csv
2025-11-15 23:51:58 - INFO - Using max token length: 256
2025-11-15 23:51:58 - INFO - Loaded 2000 records from data/test.csv
2025-11-15 23:51:58 - INFO - Using max token length: 256


In [5]:
TRAIN_NAME = config['name']
trainer = TrainPredict(TRAIN_NAME,config, label_handler, model, train_set, validation_set, test_set)
trainer.train()

2025-11-15 23:52:01 - INFO - Starting epoch 1/50
/home/gabor/kaggle/kaggle-movie-genres/.venv/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:236: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W1115 23:52:04.957000 200364 torch/_inductor/utils.py:1250] [0/0] Not enough SMs to use max_autotune_gemm mode
2025-11-15 23:53:25 - INFO - Training epoch 1 completed. Train F1 Score: 0.2996, Train Loss: 0.2715
/home/gabor/kaggle/kaggle-movie-genres/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
2025-11-15 23:53:47 - INFO - Validation epoch 1 completed. Val F1